# A Scraping Script that can scape the data from the website and store it in a csv file


In [21]:
from bs4 import BeautifulSoup
import requests
import polars as pl

ROOT_WEBSITE = "https://www.bbc.com"
NEWS_WEBSITE = "https://www.bbc.com/news"

response = requests.get(WEBSITE)
content = response.text
content_element: BeautifulSoup = BeautifulSoup(content, "lxml")
most_read_element = content_element.find("div", class_="nw-c-most-read")
most_read_links_elements_list = most_read_element.find_all("a")
df_schema = {"title": pl.Utf8, "link": pl.Utf8, "content": pl.Utf8, "image": pl.Utf8}
all_news_df = pl.DataFrame(schema=df_schema)

for most_read_links_elements in most_read_links_elements_list:
    title = most_read_links_elements.text
    link = most_read_links_elements["href"]
    new_df = pl.DataFrame(
        {"title": [title], "link": [link], "content": "None", "image": "None"}
    )
    all_news_df = all_news_df.extend(new_df)

all_news_df

title,link,content,image
str,str,str,str
"""China accused …","""/news/world-as…","""None""","""None"""
"""Binance CEO pl…","""/news/technolo…","""None""","""None"""
"""Sam Altman in …","""/news/business…","""None""","""None"""
"""King Charles d…","""/news/uk-67492…","""None""","""None"""
"""Dutch vote for…","""/news/world-eu…","""None""","""None"""
"""Canada QAnon '…","""/news/world-us…","""None""","""None"""
"""Indian captain…","""/news/world-as…","""None""","""None"""
"""Why Trump’s ri…","""/news/world-us…","""None""","""None"""
"""Teens charged …","""/news/world-us…","""None""","""None"""


In [32]:
for row in all_news_df.rows(named=True):
    link = row["link"]
    target_news_url = WEBSITE + link
    response = requests.get(target_news_url)
    content = response.text
    content_element: BeautifulSoup = BeautifulSoup(content, "lxml")
    article_element = content_element.find("article")
    image_element = article_element.find("div", {"data-component": "image-block"})
    print(article_element)
    text_element = article_element.find_all("div", {"data-component": "text-block"})
    print(text_element)

<article class="ArticleWrapper">
<header class="HeadingWrapper">
<h1 class="StyledHeading" id="main-heading" tabindex="-1">
<span class="StatusCode">404</span>
                                    Page cannot be found
                                </h1>
</header>
<div class="RichTextComponentWrapper error-message">
<p>Sorry, we're unable to bring you the page you're looking for.
                                    Please try:
                                </p>
</div>
<div class="RichTextComponentWrapper">
<div class="BulletListContainer">
<ul role="list">
<li> Double checking the url</li>
<li> Hitting the refresh button in your browser</li>
<li> Searching for this page using the BBC search bar</li>
</ul>
</div>
</div>
<p class="AlternativelyText">
                                Alternatively, please visit the
                                <a class="BbcNewsLink" href="https://www.bbc.co.uk/news">
                                    BBC News homepage.
                              